In [4]:
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import numpy as np
import os
from auto_tqdm import tqdm
import matplotlib.ticker as plticker
from typing import List
from plot_keras_history import plot_history
from glob import glob
from humanize import naturaldelta

In [5]:
def add_bar(offset:float, mean:float, std:float, color:str, label:str):
    plt.bar(offset, mean, **(
        {"yerr":std} if std>0 else {}
    ), color=color, width = 0.3, alpha=0.5, capsize=5, label=label)

In [7]:
def get_offset(bar_number:int, major_group:int, minor_group:int)->float:
    return  bar_number*0.3 + major_group + minor_group*0.25

In [8]:
def handle_label(s:int, m:int, minor_number:int, steps_number:int, offset:float, minor_group:str, major_group:str, pos:List, ticks:List):
    if s==steps_number//2:
        if steps_number % 2 == 0:
            offset -= 0.15
        pos.append(offset)
        ticks.append(
            sanitize_mode(minor_group) + ("\n\n"+sanitize_mode(major_group) if m==minor_number//2 else "")
        )

In [9]:
def hystogram(df:pd.DataFrame):
    
    fig, ax = plt.subplots(figsize=(13,5))
    ticks, pos = [], []
    colors = ["blue", "green", "red", "yellow", "cyan"]
    major_number, minor_number, steps_number = [
        len(j) for j in df.index.levels
    ]
    for i, ((c, major_group), (m, minor_group), (t, step)) in enumerate(itertools.product(*[
        list(enumerate(e)) for e in df.index.levels
    ])):
        offset = get_offset(i, c, m)
        row = df.loc[major_group, minor_group, step]
        mean = row["mean"] if "mean" in row else row.values[0]
        std = row["std"] if "std" in row else 0
        add_bar(offset, mean, std, colors[t], step)
        handle_label(t, m, minor_number, steps_number, offset, minor_group, major_group, pos, ticks)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='best')
    locs, labels = plt.yticks()
    if "required_time" in df:
        ax.set_yticklabels([
            naturaldelta(y) for y in locs
        ])
        df = df.applymap(naturaldelta)
    ax.set_xticks(pos)
    ax.set_ylabel(ylabel)
    ax.yaxis.grid(True, which="both")
    ax.set_xticklabels(ticks)
    os.makedirs("visualizations/latex", exist_ok=True)
    os.makedirs("visualizations/hystograms", exist_ok=True)
    df.columns = [
        c.replace("_", " ") for c in df.columns
    ]
    df = df.unstack(level=1).reorder_levels([1,0], axis=1).sort_index(axis=1, sort_remaining=False)
    df.columns.names = ["" for _ in df.columns.names]
    df.to_latex(
        "visualizations/latex/{title}.tex".format(title=title.lower().replace(" ", "_")),
        float_format=lambda x: '%.3f' % x,
        bold_rows=True
    )
    plt.title(title.replace("_", " "))
    plt.tight_layout()
    plt.savefig("visualizations/hystograms/{title}.png".format(title=title.lower().replace(" ", "_")))
    plt.show()
    plt.close()

In [11]:
sum([True, True, False])

2